In [ ]:
import cvxpy
import numpy as np
import random as rm
import random
from scipy.stats import poisson
from scipy.stats import expon
from re import X

#np.random.seed(seed=1)

class MailroomSim:
    #m/m/1 queue with exponential distribution (mean 1) for arrival and service times
    def __init__(self):

        self.arrival_times_list = []
        self.newDeparturesList1 = []
        self.newDeparturesList2 = []
        self.waitTimes = []

        self.lamb = 1
        self.mu = 0.9


    def increment_time(self): 
        """
        Creates output of all arrival and departure times within 8am-10pm Monday-Friday business hours
        Generates arrival times in an exponential distrbution 
        """
        count_time = 0.0

        random_exp_list = expon.rvs(scale = self.lamb, size = 10000)
        while count_time <= 50000:
            count_time += np.random.choice(random_exp_list)
            #For all values in the randomly generated list, adds the values to count time
            if count_time <= 840:
                self.arrival_times_list.append(count_time)    
            #print(count_time)


          ## mm2
        lastDeparture1 = 0.0
        lastDeparture2 = 0.0
        currentClock = 0.0

        count = 1
        for x in self.arrival_times_list:
            minDeparture = min(lastDeparture1, lastDeparture2)
            maxDeparture = max(lastDeparture1, lastDeparture2)
            
            #print(f"T: {count:3} | x: {x:6.2f} | LD1:{lastDeparture1:6.2f} | LD2:{lastDeparture2:6.2f}")
            count+=1
            
            # both two lanes are available
            if x >= maxDeparture:
                lastDeparture1 = x + self.service_time() # assign to the first one or can random pick one
                self.newDeparturesList1.append(lastDeparture1)
                #print(f"--->b1             | LD1:{lastDeparture1:6.2f} | LD2:{lastDeparture2:6.2f}")

            # only one lane is available
            elif x >= minDeparture:
                if lastDeparture1 < lastDeparture2: # 1 is available but 2 is not
                    lastDeparture1 = x + self.service_time() 
                    self.newDeparturesList1.append(lastDeparture1)
                    #print(f"--->b2.1           | LD1:{lastDeparture1:6.2f} | LD2:{lastDeparture2:6.2f}")
                else: # 2 is available but 1 is not
                    lastDeparture2 = x + self.service_time() 
                    self.newDeparturesList2.append(lastDeparture2)
                    #print(f"--->b2.2           | LD1:{lastDeparture1:6.2f} | LD2:{lastDeparture2:6.2f}")

            # neither lanes are available
            else: 
                self.waitTimes.append(minDeparture - x) 
                if lastDeparture1 < lastDeparture2:
                    lastDeparture1 = lastDeparture1 + self.service_time()
                    self.newDeparturesList1.append(lastDeparture1)
                    #print(f"--->b3.1           | LD1:{lastDeparture1:6.2f} | LD2:{lastDeparture2:6.2f}")
                else: 
                    lastDeparture2 = lastDeparture2 + self.service_time() 
                    self.newDeparturesList2.append(lastDeparture2)
                    #print(f"--->b3.2           | LD1:{lastDeparture1:6.2f} | LD2:{lastDeparture2:6.2f}")

#         sum = 0
#         count = 0
#         for i in range(len(self.newDeparturesList)):
#             waitTime = self.newDeparturesList[i] - self.arrival_times_list[i]
#             sum+= waitTime
#             count+=1
#             self.waitTimes.append(waitTime)

#         average = sum/count

        print("Arrivals:")
        print(self.arrival_times_list)
        print("Departures1:")
        print(self.newDeparturesList1)
        print("Departures2:")
        print(self.newDeparturesList2)
        print("Wait Times:")
        print(self.waitTimes)
        print("average wait time:")
        print(sum(self.waitTimes)/len(self.waitTimes))


    #how long something takes to process depending on mail or other request (values can be modified based on observed real data)
    def service_time(self):
        service_time = np.random.exponential(1/self.mu)
#         print(f'service time: {service_time:.2f}')
        return service_time
      
s = MailroomSim()
s.increment_time()

Arrivals:
[1.29542971534631, 2.943095767263119, 3.040008750045905, 3.2541334559070836, 3.5793179200465506, 4.4631883338720275, 4.798992477860453, 5.644013066567793, 7.040072626130958, 8.923033778409348, 11.556944058308444, 12.488528027661548, 13.612517189872875, 13.750000097958406, 14.132554434857802, 18.922227690573308, 19.10070302017533, 22.98679354364858, 27.012741134391327, 27.321797454390584, 27.646898877000137, 28.09550089709454, 28.62036837300757, 29.230220413131892, 29.494304798987812, 30.451867725777447, 30.920184176731244, 32.83521165599207, 33.430481122097135, 36.05847724561171, 37.33216325857654, 37.366923149927686, 38.72931473287831, 41.14147731235537, 41.91747715708583, 43.07816239979882, 44.13815012957795, 44.9863385402748, 45.75099780042929, 46.33445869717664, 47.77662681803403, 48.307186959784644, 49.344503680189376, 54.39757474296182, 54.9236818202938, 57.007643506381356, 57.753659921021125, 60.64366486424026, 61.49166583998804, 62.411884533000446, 62.46401098626861, 

In [ ]:
def generate_packages(env, arrival_rate_small, arrival_rate_large, service_time_small, service_time_large, server):
    # Loop to generate packages
    while True:
        # Generate a small package
        if random.random() < arrival_rate_small:
            package_size = 'small'
            service_time = random.expovariate(1 / service_time_small)
        # Generate a large package
        elif random.random() < arrival_rate_small + arrival_rate_large:
            package_size = 'large'
            service_time = random.expovariate(1 / service_time_large)
        # No package arrives
        else:
            yield env.timeout(1)
            continue

        # Add package to queue
        with server.request() as req:
            yield req

            # Process package
            yield env.timeout(service_time)
